In [1]:
from lxml import etree

In [2]:
from typing import List, Tuple

In [3]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [4]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [5]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [6]:
import random

In [7]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

negative => Сбербанк, спасибо за напоминание, но я не брала у вас кредит.
negative => Вообще, конечно, багрепорт надо было в сбер отправлять, а не публиковать – но со сбером это не особо поможет, хотя
neutral => RT @qipygucagek: банк москвы краснодар автокредит http://t.co/qALV4jcjO9
neutral => Банки москвы кредиты райфайзен банк http://t.co/FxT9fZDCJZ
neutral => банк москвы калькулятор кредита автокредит http://t.co/OQb07lffuV
neutral => Сбербанк ведет переговоры о производстве Fiat и Renault на базе ЗИЛа
positive => По состоянию на 10 часов утра 24 июня рейтинг банков Бурятии, за которые голосуют посетители сайта «Байкал Финанс», возглавил Сбербанк Росс
neutral => @shtorkin я через мобильный банк оплачиваю, сбер не спрашивает оператора
neutral => бланк доверенности в сбербанк на получение документов http://t.co/wY6AUyGXHG #бланк #доверенности
neutral => RT @xlerMauzy: #Как Оформить Кредит В Сбер Банк Россиии http://t.co/NGMcJomALe
neutral => альфабанк оплатить кредит http://t.co/XNwq

In [8]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']

In [9]:
for cur in positive_tweets[:5]: print(cur)

@sawik_shuster @YevhenS Главное чтоб банки СБЕР и ВТБ!!!
Самый выгодный автокредит в втб 24 http://t.co/CDJ5P40x7g
http://t.co/h6r6GdBe4H Легко можно получить денежный кредит ы втб 24 банке
Снижение процентной ставки по кредиту на недвижимость сбербанк http://t.co/MredO898wK
В наш сбербанк пришел американец и попросил сфоткаться с ним у банкомата ;D 
я ваще была в шоке и на фотке полюбому получилась смешная xD


In [10]:
for cur in negative_tweets[:5]: print(cur)

@ShtirliZ_ @Zhukova_olga @winzard @tereshenkov обязательно про сбербанк напишите! Временами похлеще интернет магазинов финты выкручивает)
@Umka_75 @Alfarius2012 @igor_sechin втб и сбер точно. вопрос-блокировка операций или запрет на кредиты?
Канада ввела санкции против Газпромбанка, ВЭБа, Новатэка и российских оборонных компаний
RT @letokot: Если Сбер и ВТБ попали под санкции, будет жопа
RT @360_tv_ru: #Канада ввела санкции против 10 оборонных и сырьевых компаний и банков #РФ. Среди них, в частности, "#Газпромбанк", "#Внешэк…


In [11]:
from nltk import word_tokenize

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [14]:
vectorizer.fit(texts)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x0000013530FB88B0>)

In [15]:
print(vectorizer.get_feature_names()[0:20])

['!', '#', '$', '%', '&', "''", '(', ')', '*', '+', '+3,49', '+300', '+5тыс', '+74955102933', '+в', ',', '-', '--', '-4.8', '-5']


In [16]:
print(len(vectorizer.get_feature_names()))

20192


In [17]:
X = vectorizer.transform(texts)

In [18]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [19]:
print(texts[0])

http://t.co/YEVHuvVGA1 Взять кредит тюмень альфа банк


In [20]:
print(X[0])

  (0, 6318)	1
  (0, 7196)	1
  (0, 7773)	1
  (0, 9092)	1
  (0, 9293)	1
  (0, 9861)	1
  (0, 12787)	1
  (0, 19011)	1


In [21]:
print(vectorizer.get_feature_names()[6321])

//t.co/yfqcav48qj


In [22]:
print(vectorizer.get_feature_names()[9866])

видать


In [23]:
print(vectorizer.get_feature_names()[19056])

уверил


In [24]:
from sklearn.feature_extraction.text import TfidfTransformer

In [25]:
transformer = TfidfTransformer().fit(X)

In [26]:
X_transformed = transformer.transform(X)

In [27]:
print(X_transformed[0])

  (0, 19011)	0.5196767551267244
  (0, 12787)	0.17477128797745156
  (0, 9861)	0.2815132193133801
  (0, 9293)	0.20528209436889633
  (0, 9092)	0.26133550987375115
  (0, 7773)	0.10105340406350267
  (0, 7196)	0.09241786572867976
  (0, 6318)	0.7006793382592054


In [28]:
print(vectorizer.get_feature_names()[19056])

уверил


In [29]:
print(vectorizer.get_feature_names()[7199])

=


In [30]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [31]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
3.144655 => #
6.276519 => $
5.636861 => %
5.647910 => &
4.134005 => ''
4.805386 => (
4.443938 => )
8.201810 => *
9.049108 => +
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тыс
9.454573 => +74955102933
9.454573 => +в
2.753227 => ,
4.420872 => -
8.761426 => --
9.454573 => -4.8
9.454573 => -5


In [32]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [33]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => +3,49
9.454573 => +300
9.454573 => +5тыс
9.454573 => +74955102933
9.454573 => +в
9.454573 => -4.8
9.454573 => -5
9.454573 => -6.4
9.454573 => -а
9.454573 => -круто
9.454573 => -осуществление
9.454573 => -подключение
9.454573 => -привлечение
9.454573 => -разменяйте
9.454573 => -россельхозбанка
9.454573 => -сами
9.454573 => -сервис
9.454573 => -сын
9.454573 => -то
9.454573 => -филиал


In [34]:
from sklearn.feature_selection import SelectPercentile, chi2

In [35]:
selector = SelectPercentile(chi2, percentile=20)

In [36]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20,
                 score_func=<function chi2 at 0x00000135320F3280>)

In [37]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [38]:
print(len(selected_tokens_with_IDF))

4039


In [39]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
3.144655 => #
5.636861 => %
4.134005 => ''
4.805386 => (
4.443938 => )
8.201810 => *
9.049108 => +
9.454573 => +5тыс
2.753227 => ,
4.420872 => -
9.454573 => -россельхозбанка
3.189272 => .
5.465589 => ..
3.819783 => ...
7.103198 => ....
9.454573 => ._
6.070183 => /
9.454573 => //t.co/01ftjc0bur
9.454573 => //t.co/0gfmj3s0gi


In [40]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [41]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => +5тыс
9.454573 => -россельхозбанка
9.454573 => ._
9.454573 => //t.co/01ftjc0bur
9.454573 => //t.co/0gfmj3s0gi
9.454573 => //t.co/0i4pf81ksd
9.454573 => //t.co/0jzv0jq959
9.454573 => //t.co/0lt5jdmk9l
9.454573 => //t.co/0s03a0v7po
9.454573 => //t.co/0ttwpnjnef
9.454573 => //t.co/0vpcvumoe8
9.454573 => //t.co/0wziwezr91
9.454573 => //t.co/0y05qjok2q
9.454573 => //t.co/1aa3jpbxqh
9.454573 => //t.co/1cjshtzrda
9.454573 => //t.co/1d3bb71qbt
9.454573 => //t.co/1eodyxqq0a
9.454573 => //t.co/1fxzerzsps
9.454573 => //t.co/1kiwdyic6t
9.454573 => //t.co/1mymy2lxfu


In [ ]:
#1)токенизацию с учётом возможных смайлов (базовая токенизация некорректно
#работает со знаками препинания и прочими неалфавитными и нецифровыми
#символами, из которых как раз и могут состоять тонально значимые смайлы)

In [42]:
from nltk.tokenize import TweetTokenizer

In [43]:
tokenizer = TweetTokenizer()

In [44]:
def drop_junk(s):
    return ' '.join([word for word in tokenizer.tokenize(s) if word.isalpha() or '!' in word or '?' in word \
                     or '(' in word or ')' in word])

In [45]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [53]:
texts = list(map(drop_junk, texts))


In [ ]:
# 2) лемматизацию с учётом контекста, чтобы успешно разрешать морфоомонимию вида
# “​ мы ​ стали лучше программировать​ ” - “​ мы выплавляем больше ​ стали​ ” (для такой
# лемматиции можно использовать, например, библиотеку ​ UDPipe или её адаптацию
# под ​ SpaCy​ )

In [57]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language='ru')

In [58]:
def lemmatization(s):
    toks = tokenizer.tokenize(s)
    forms = predictor.predict(toks)
    return ' '.join([f.normal_form for f in forms])


In [59]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [60]:
texts = list(map(lemmatization, texts))

In [ ]:
# 3) удаление стоп-слов по словарям и/или правилам (например, описанным в виде
# регулярных выражений).

In [79]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
from nltk.corpus import stopwords 

In [81]:
stop_words = get_stop_words('ru')
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize, stop_words=stop_words)

In [82]:
vectorizer.fit(texts)

CountVectorizer(stop_words=['а', 'в', 'г', 'е', 'ж', 'и', 'к', 'м', 'о', 'с',
                            'т', 'у', 'я', 'бы', 'во', 'вы', 'да', 'до', 'ее',
                            'ей', 'ею', 'её', 'же', 'за', 'из', 'им', 'их',
                            'ли', 'мы', 'на', ...],
                tokenizer=<function word_tokenize at 0x0000013530FB88B0>)